## Importamos las librerias 

In [1]:
import numpy as np 
import json
import re
import warnings
import random 
import spacy

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow import expand_dims

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

nlp= spacy.load('es_core_news_lg') # carga un modelo de lenguaje de spaCy en la variable nlp. El modelo 'en_core_web_sm' es un modelo de lenguaje en Español

warnings.filterwarnings('ignore') # suprime todos los mensajes de advertencia que se generan durante la ejecución del código. Esto puede ser útil si estás ejecutando un código que genera muchos mensajes de advertencia que no son relevantes para tu tarea.

## Leemos y Procesamos nuestros datos

In [2]:
def pre_processing(line):
    re.sub(r'[^a-záéíóúñüÁÉÍÓÚÑÜ0-9,.!?;]', " ", line)
    #line= re.sub(r'[]+',' ', line)
    return line

In [3]:
# Leer datos

with open('intentos_pizzas.json','rb') as file:
    data= json.load(file)
    # obtener texto y títulos de intención a partir de datos json
    inputs, targets= [], []
    cls= []
    intent_doc= {}

    for i in data['intents']:
        if i['intent'] not in cls:
            cls.append(i['intent'])
        
        if i['intent'] not in intent_doc:
            intent_doc[i['intent']]= []
        
        for text in i['text']:
            inputs.append(pre_processing(text))
            targets.append(i['intent'])
        
        for response in i['responses']:
            intent_doc[i['intent']].append(response)


In [4]:
print(inputs)
print(targets)
print(cls)

['Hola', 'Buenas', 'Hola, ¿cómo estás?', 'Buenas tardes', 'Buenas noches', 'Hola tienes servicio', 'tienes comida', 'Tienes servicio', 'Quiero ver el menú', 'Que tienes de comer', 'Qué opciones tienen?', '¿Puedes mostrarme la carta?', 'Menú por favor', '¿Qué puedo ordenar?', 'Necesito ver las opciones', 'que tienes de comer', 'Tienes carta', 'Que comida tienes', 'Que me ofreces', 'Que tienes de comida', 'Que perros tienes', 'De que tienes los perros', 'cuales perros me ofreces', 'como cuales perros tienes', 'Que tipo de perros tienes', 'Que salchipapas tienes', 'De que tienes las salchipapa', 'cuales salchipapa me ofreces', 'como cuales salchipapa tienes', 'Que tipo de salchipapa tienes', 'Que hamburguesas tienes', 'De que tienes las hamburguesas', 'cuales hamburguesas me ofreces', 'como cuales hamburguesas tienes', 'Que tipo de hamburguesas tienes', 'Quiero ordenar una Pizza Megafamiliar y dos porciones de pizzas', 'Me gustaría una Pizza Familiar y 4 pizza por porcion', 'Quiero una Pi

In [5]:
def cr_cat_targe(targets):
    word= {}
    cat_t= []
    counter= 0
    
    for trg in targets:
        if trg not in word:
            word[trg]= counter
            counter+=1
        cat_t.append(word[trg])
        
    cat_tensor= to_categorical(cat_t, num_classes=len(word), dtype='int32')
    return cat_tensor, dict((v,k) for k,v in word.items())

target_tensor, target_indx_word= cr_cat_targe(targets)

In [16]:
print(target_indx_word)
print(target_tensor)

{0: 'Saludo', 1: 'PedidoCarta', 2: 'Clasedeperros', 3: 'Clasedesalchipapa', 4: 'ClasedeHamburguesas', 5: 'RealizarPedido', 6: 'ConfirmarPedido', 7: 'AgregarQuitarPedido', 8: 'IngredientesPizza', 9: 'IngredientesEspecificosCriolla', 10: 'IngredientesEspecificosPollo', 11: 'IngredientesEspecificoscarnes', 12: 'PorcionesPizzaMG', 13: 'PorcionesPizzaF', 14: 'ProporcionarDireccion'}
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [7]:
def token_data(inp_list):
    tokenizer= Tokenizer(filters='', oov_token='<unk>')
    tokenizer.fit_on_texts(inp_list)
    inp_seq= tokenizer.texts_to_sequences(inp_list)
    
    # Agregando padding 
    inp_seq= pad_sequences(inp_seq, padding='pre')
    return tokenizer, inp_seq

tokenizer, inp_tensor= token_data(inputs)

In [8]:
print(inp_tensor)

[[  0   0   0 ...   0   0  47]
 [  0   0   0 ...   0   0  36]
 [  0   0   0 ...  73  74  75]
 ...
 [  0   0   0 ...   2  45  58]
 [  0   0   0 ...  46 124 125]
 [  5  35  72 ... 129  71  69]]


## Creamos nuestra red neuronal

In [9]:
# Paramestros de nuestros modelo 
epochs= 50
vocab_size= len(tokenizer.word_index)+1
embed_dim= 512
units= 120
target_len= target_tensor.shape[1]

In [10]:
# Modelo 
modelo= Sequential([
    Embedding(vocab_size, embed_dim),
    Bidirectional(LSTM(units, dropout=0.2)),
    Dense(units, activation= 'relu'),
    Dropout(0.5),
    Dense(target_len, activation='softmax')
])

In [11]:
modelo.compile(optimizer=Adam(lr= 1e-2), loss= 'categorical_crossentropy', metrics=['accuracy'])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         66560     
                                                                 
 bidirectional (Bidirection  (None, 240)               607680    
 al)                                                             
                                                                 
 dense (Dense)               (None, 120)               28920     
                                                                 
 dropout (Dropout)           (None, 120)               0         
                                                                 
 dense_1 (Dense)             (None, 15)                1815      
                                                                 
Total params: 704975 (2.69 MB)
Trainable params: 704975 (2.69 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [12]:
# nuestra red se dentendra en el mejor entrenamiento
early_spot= EarlyStopping(monitor='loss', patience= 5) # EarlyStopping que espere 2 épocas sin mejora en la pérdida antes de detener el entrenamiento

modelo.fit(inp_tensor, target_tensor, epochs= epochs, callbacks=[early_spot])

Epoch 1/50
3/3 [==============================] - 11s 59ms/step - loss: 2.7076 - accuracy: 0.1034
Epoch 2/50
3/3 [==============================] - 0s 53ms/step - loss: 2.6595 - accuracy: 0.1149
Epoch 3/50
3/3 [==============================] - 0s 52ms/step - loss: 2.6100 - accuracy: 0.1379
Epoch 4/50
3/3 [==============================] - 0s 57ms/step - loss: 2.5424 - accuracy: 0.1264
Epoch 5/50
3/3 [==============================] - 0s 55ms/step - loss: 2.4597 - accuracy: 0.1149
Epoch 6/50
3/3 [==============================] - 0s 54ms/step - loss: 2.4409 - accuracy: 0.1839
Epoch 7/50
3/3 [==============================] - 0s 58ms/step - loss: 2.3734 - accuracy: 0.2529
Epoch 8/50
3/3 [==============================] - 0s 52ms/step - loss: 2.2524 - accuracy: 0.3448
Epoch 9/50
3/3 [==============================] - 0s 58ms/step - loss: 2.2547 - accuracy: 0.3448
Epoch 10/50
3/3 [==============================] - 0s 61ms/step - loss: 2.0558 - accuracy: 0.3563
Epoch 11/50
3/3 [===========

## Creamos nuestra funcion de respuesta


In [13]:
def response(sentence):
    sent_seq= []
    doc= nlp(repr(sentence))
    
    # split the input sentences into words
    
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])
            
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])
            
    sent_seq= expand_dims(sent_seq, 0)
    
    # predict the category of input sentences
    pred= modelo(sent_seq)
    pred_class= np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[target_indx_word[pred_class[0]]]), target_indx_word[pred_class[0]]

In [14]:
response('quiero 2 pizzas')

('Entendido. Para confirmar, has pedido:\n- <Cantidad> <Producto>\n¿Es correcto?',
 'RealizarPedido')

## Simulacion con el chat bot 

In [15]:
print("Note: Enter 'quit' to breck the loop")

while True:
    respuesta = input('Tú: ')
    if respuesta.lower() == 'quit':
        break

    res, typ = response(respuesta)
    print('Bot:{}--TYPE: {}'.format(res, typ))
    print()

Note: Enter 'quit' to breck the loop
